In [5]:
# Naive Bayes Algorithm
zeroCorrection = False # True
nFeature = 3

#print '**********'

In [4]:
featureNames = []
# read in training dataset
trainingInstances = []
fr = open('train.txt','rb')
line = fr.readline()
arr = line.strip('\r\n').split('\t')
for i in range(nFeature):
    featureNames.append(arr[i+3])
for line in fr:
    arr = line.strip('\r\n').split('\t')
    featureValues = arr[3:nFeature+3]
    label = arr[nFeature+3]
    trainingInstances.append([featureValues,label])
fr.close()

In [6]:
# zero correction if the probability of a instance is 0 in training dataset
if zeroCorrection:
    import itertools

    valuesetList = [set() for i in range(nFeature)]
    labelset = set()
    for [featureValues,label] in trainingInstances:
        for i in range(nFeature):
            valuesetList[i].add(featureValues[i])
        labelset.add(label)
    values_labels = []
    for i in range(nFeature):
        values_labels.append(list(valuesetList[i]))
    values_labels.append(list(labelset))
    _product = itertools.product(values_labels[0],values_labels[1])
    for i in range(2,nFeature+1):
        _product = itertools.product(_product,values_labels[i])
    for x in _product:
        label = x[-1]
        featureValues = []
        for i in range(nFeature-1):
            x = x[0]
            featureValues.insert(0,x[-1])
        featureValues.insert(0,x[0])
        trainingInstances.append([featureValues,label])


In [12]:
print 'Training data:'
nTrainingInstance = len(trainingInstances)
for i in range(nTrainingInstance):
    print i+1,trainingInstances[i]


Training data:
1 [['Home', 'Out', 'NBC'], 'Win']
2 [['Away', 'Out', 'ABC'], 'Win']
3 [['Home', 'In', 'NBC'], 'Win']
4 [['Home', 'Out', 'NBC'], 'Win']
5 [['Away', 'In', 'ABC'], 'Lose']
6 [['Home', 'Out', 'NBC'], 'Win']
7 [['Home', 'In', 'NBC'], 'Win']
8 [['Away', 'Out', 'ABC'], 'Win']
9 [['Away', 'Out', 'ABC'], 'Win']
10 [['Home', 'Out', 'NBC'], 'Win']
11 [['Away', 'Out', 'NBC'], 'Win']
12 [['Away', 'In', 'FOX'], 'Lose']
13 [['Away', 'Out', 'ABC'], 'Lose']
14 [['Home', 'Out', 'NBC'], 'Win']
15 [['Home', 'Out', 'NBC'], 'Lose']
16 [['Home', 'Out', 'NBC'], 'Lose']
17 [['Home', 'Out', 'ESPN'], 'Win']
18 [['Away', 'Out', 'ABC'], 'Lose']
19 [['Home', 'In', 'NBC'], 'Lose']
20 [['Home', 'Out', 'NBC'], 'Win']
21 [['Home', 'Out', 'CBS'], 'Lose']
22 [['Home', 'Out', 'NBC'], 'Win']
23 [['Home', 'In', 'NBC'], 'Lose']
24 [['Away', 'In', 'ABC'], 'Lose']


In [8]:
# probability calculation
label2count = {}
label2value2count_list = [{} for i in range(nFeature)]
for instanceID in range(nTrainingInstance):
    featureValues,label = trainingInstances[instanceID]
    if not label in label2count:
        label2count[label] = 0
    label2count[label] += 1
    for featureID in range(nFeature):
        value = featureValues[featureID]        
        if not label in label2value2count_list[featureID]:
            label2value2count_list[featureID][label] = {}
        if not value in label2value2count_list[featureID][label]:
            label2value2count_list[featureID][label][value] = 0
        label2value2count_list[featureID][label][value] += 1

label2prob = {} # P(X)
for [label,count] in label2count.items():
    label2prob[label] = 1.0*count/nTrainingInstance

label2value2prob_list = [{} for i in range(nFeature)]  # P(X|H)
for featureID in range(nFeature):
    for label in label2prob:
        label2value2prob_list[featureID][label] = {}
        totalCount = 0
        for [value,count] in label2value2count_list[featureID][label].items():
            totalCount += count
        for [value,count] in label2value2count_list[featureID][label].items():
            prob = 1.0*count/totalCount
            label2value2prob_list[featureID][label][value] = prob


In [11]:
# Evaluation Process start 
def Predict(trainingInstances,featureValues):
    label2posteriori = {}
    for [label,prob] in label2prob.items():
        posteriori = prob
        for featureID in range(nFeature):
            if not label in label2value2prob_list[featureID]:
                posteriori = 0
                break
            value = featureValues[featureID]
            if not value in label2value2prob_list[featureID][label]:
                posteriori = 0
                break
            posteriori *= label2value2prob_list[featureID][label][value]
        label2posteriori[label] = posteriori
    label_posteriori = sorted(label2posteriori.items(),key=lambda x:-x[1])
    return label_posteriori[0][0]

testInstances = []
fr = open('test.txt','rb')
line = fr.readline()
for line in fr:
    arr = line.strip('\r\n').split('\t')
    featureValues = arr[3:nFeature+3]
    label = arr[nFeature+3]
    testInstances.append([featureValues,label])
fr.close()

TP,FP,TN,FN = 0,0,0,0

print 'Test data:'
nTestInstance = len(testInstances)
for i in range(nTestInstance):
    featureValues = testInstances[i][0]
    truthLabel = testInstances[i][1]
    predictLabel = Predict(trainingInstances,featureValues)
    print i+1+nTrainingInstance,featureValues,'ground-truth:',truthLabel,'predict:',predictLabel
    if truthLabel == 'Win' and predictLabel == 'Win': TP += 1
    if truthLabel == 'Lose' and predictLabel == 'Win': FP += 1
    if truthLabel == 'Win' and predictLabel == 'Lose': FN += 1
    if truthLabel == 'Lose' and predictLabel == 'Lose': TN += 1


Test data:
25 ['Home', 'Out', 'NBC'] ground-truth: Win predict: Win
26 ['Home', 'In', 'NBC'] ground-truth: Lose predict: Win
27 ['Away', 'Out', 'ESPN'] ground-truth: Win predict: Win
28 ['Away', 'Out', 'FOX'] ground-truth: Win predict: Lose
29 ['Home', 'Out', 'NBC'] ground-truth: Win predict: Win
30 ['Away', 'Out', 'ABC'] ground-truth: Win predict: Lose
31 ['Home', 'In', 'NBC'] ground-truth: Win predict: Win
32 ['Home', 'Out', 'NBC'] ground-truth: Win predict: Win
33 ['Home', 'Out', 'NBC'] ground-truth: Win predict: Win
34 ['Away', 'In', 'ABC'] ground-truth: Lose predict: Lose
35 ['Home', 'Out', 'NBC'] ground-truth: Win predict: Win
36 ['Away', 'In', 'ABC'] ground-truth: Lose predict: Lose


In [10]:
print 'Confusion matrix:'
print '---------------'
print '| TP',TP,'|','FN',FN,'|'
print '| FP',FP,'|','TN',TN,'|'
print '---------------'

accuracy = 1.0*(TP+TN)/nTestInstance
precision = 0.0
if TP+FP > 0: precision = 1.0*TP/(TP+FP)
recall = 0.0
if TP+FN > 0: recall = 1.0*TP/(TP+FN)
f1 = 0.0
if precision > 0.0 and recall > 0.0: f1 = 2*precision*recall/(precision+recall)

print 'Accuracy:',accuracy
print 'Precision:',precision
print 'Recall:',recall
print 'F1:',f1

Confusion matrix:
---------------
| TP 7 | FN 2 |
| FP 1 | TN 2 |
---------------
Accuracy: 0.75
Precision: 0.875
Recall: 0.777777777778
F1: 0.823529411765
